Librerías

In [1]:
import json
import pandas as pd
import mysql.connector
from mysql.connector import Error

Funciones

In [3]:
#Conexión instancia
def create_connection(json_host_file):
    connection = None
    try:
        with open(json_host_file) as json_file:
            data = json.load(json_file)
            host_name=data['host_name']
            user_name=data['user_name']
            user_password=data['user_password']
            db_name=data['db_name']

        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

# INSERT / UPDATE / DELETE
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

# SELECT (OTRA ALTERNATIVA)
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    column_names = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        column_names =  cursor.column_names
        return [result, column_names]
    except Error as e:
        print(f"The error '{e}' occurred")


Prueba SELECT/INSERT/UPDATE/DELETE

In [4]:
#Conexion con BD usando archivo json
connection = create_connection('db_conection.json')

Connection to MySQL DB successful


In [5]:
#SELECT

table = "area"

q0 = f"""
SELECT * FROM {table};
"""

pd.read_sql_query(q0, connection)

,id_area,area,tipo,jefatura,objetivo,descripcion


In [6]:
#INSERT

table = "area"
columns = "(id_area, area, tipo, jefatura, objetivo, descripcion)"
values1 =  "(1, 'area 1', 'Oficina', '11.111.111-1', 'objetivo 1', 'descripcion')"
values2 =  "(2, 'area 2', 'Gerencia', '22.222.222-2', 'objetivo 2', 'descripcion')"

q1 = f"""
INSERT INTO {table} {columns}
VALUES {values1}, {values2};
"""

execute_query(connection, q1)
pd.read_sql_query(q0, connection)

Query executed successfully


,id_area,area,tipo,jefatura,objetivo,descripcion
0,1,area 1,Oficina,11.111.111-1,objetivo 1,descripcion
1,2,area 2,Gerencia,22.222.222-2,objetivo 2,descripcion


In [8]:
#UPDATE

table = "area"

q2 = f""" 
UPDATE {table} 
SET descripcion = 'descripción area 1'
WHERE id_area  = 1;"""

q3 = f""" 
UPDATE {table} 
SET objetivo = 'Objetivo area 2'
WHERE id_area  = 2;"""

execute_query(connection, q2)
execute_query(connection, q3)
pd.read_sql_query(q0, connection)

Query executed successfully
Query executed successfully


,id_area,area,tipo,jefatura,objetivo,descripcion
0,1,area 1,Oficina,11.111.111-1,objetivo 1,descripción area 1
1,2,area 2,Gerencia,22.222.222-2,Objetivo area 2,descripcion


In [11]:
#DELETE

table = "area"

q4 = f""" 
DELETE FROM {table}
WHERE id_area  IN (1, 2);"""

execute_query(connection, q4)
pd.read_sql_query(q0, connection)

Query executed successfully


,id_area,area,tipo,jefatura,objetivo,descripcion
